In [1]:
import numpy as np
import pandas
import torch
import torch.nn as nn
from model.decoder import Decoder
from model.encoder import Encoder
from model.autoencoder import AE

In [2]:
ae_pretrain = AE(Encoder, Decoder)

In [3]:
ae_pretrain.load_state_dict(torch.load('./logging/model.pth'))

<All keys matched successfully>

In [4]:
from utils.data_loading import CombustionSystemDataset

PATH = 'Zare_data/combustion_img_13.mat'

train_data = CombustionSystemDataset(PATH,'train_set_x','train_set_y')
test_data = CombustionSystemDataset(PATH,'test_set_x','test_set_y')

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=6, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=6, shuffle=False)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ae_pretrain.to(device)

AE(
  (encoder): Encoder(
    (conv_layers): Sequential(
      (0): Conv2d(1, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
      (5): ReLU()
    )
    (dense_layers): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=46080, out_features=512, bias=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=512, out_features=1024, bias=True)
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (decoder): Decoder(
    (conv_layers): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), output_padding=(1, 1))
      (1): ReLU()
      (2): ConvTranspose2d(64, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 0))
      (3): ReLU()
      (4): ConvTranspose2d(32, 1, kernel_size=(5, 3), s

In [7]:
batch_ct=0   #Diagnositic
for train_images, train_targets in train_loader:
    batch_ct=batch_ct+1
    train_images, train_targets = train_images.to(device), train_targets.to(device)
    train_codes = ae_pretrain.encoder(train_images)
  
    df_code = pandas.DataFrame(train_codes.to('cpu').detach().numpy())
    df_class=pandas.DataFrame(train_targets.to('cpu').detach().numpy())
    df_code.to_csv('train_embbeds.csv', mode='a', index=False, header=False)
    df_class.to_csv('train_labels.csv', mode='a', index=False, header=False)

    if batch_ct % 1000 == 0:
        print("Batch: ",batch_ct)

Batch:  1000
Batch:  2000
Batch:  3000
Batch:  4000
Batch:  5000
Batch:  6000
Batch:  7000
Batch:  8000
Batch:  9000


In [8]:
batch_ct=0 
for test_images, test_targets in test_loader:
    
    test_images, test_targets = test_images.to(device), test_targets.to(device)
    test_codes = ae_pretrain.encoder(test_images)
  
    df_code = pandas.DataFrame(test_codes.to('cpu').detach().numpy())
    df_class=pandas.DataFrame(test_targets.to('cpu').detach().numpy())
    df_code.to_csv('test_embbeds.csv', mode='a', index=False, header=False)
    df_class.to_csv('test_labels.csv', mode='a', index=False, header=False)

    batch_ct=batch_ct+1
    if batch_ct % 1000 == 0:
        print("Batch: ",batch_ct)

Batch:  1000
Batch:  2000
Batch:  3000


In [9]:
valid_data = CombustionSystemDataset(PATH,'valid_set_x','valid_set_y')
valid_loader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=6, shuffle=False)

In [10]:
batch_ct=0   #Diagnositic
for valid_images, valid_targets in valid_loader:
    
    valid_images, valid_targets = valid_images.to(device), valid_targets.to(device)
    valid_codes = ae_pretrain.encoder(valid_images)
  
    df_code = pandas.DataFrame(valid_codes.to('cpu').detach().numpy())
    df_class = pandas.DataFrame(valid_targets.to('cpu').detach().numpy())
    df_code.to_csv('valid_embbeds.csv', mode='a', index=False, header=False)
    df_class.to_csv('valid_labels.csv', mode='a', index=False, header=False)

    batch_ct=batch_ct+1
    if batch_ct % 1000 == 0:
        print("Batch: ",batch_ct)

Batch:  1000
